## Import libraries
Developing in Jupyter Lab, so using JupyterDash

In [1]:
import pandas as pd
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import censoring

## Configure for JupyterDash

In [3]:
JupyterDash.infer_jupyter_proxy_config()

## Filter

In [4]:
df = pd.read_csv(censoring.wordle_output)
# limit to the puzzle in which we last have words (396)
df2 = df[ (df['PuzzleNum'] < 467) & 
         (df['PuzzleNum'] != 421) & 
         (df['Difficulty'] != 'Undefined') &
         (df['Name'] != 'Player7') & 
         (df['Name'] != 'Player8')
        ].copy()

# limit puzzles to where 6 players participated
df3 = df2[['Name','PuzzleNum']].groupby(['PuzzleNum']).agg('count').reset_index()
list_of_puzzles = df3[ df3['Name'] == 6 ]['PuzzleNum'].tolist()
df2 = df2[ df2['PuzzleNum'].isin(list_of_puzzles)].copy()

# make player list
player_list = df2['Name'].unique().tolist()

# make sure date time is correct type
df2['Date_Time']= pd.to_datetime(df2['Date_Time'])
df2['WeekNum'] = df2['Date_Time'].dt.isocalendar().week

# static ranking data frame
ranking_df = df2.groupby(['Name']).agg({'Fails':'sum'}).reset_index()
ranking_df['MeanWeeklyFails'] = ranking_df['Fails'] / df2['WeekNum'].max()
ranking_df = ranking_df.sort_values(by=['Fails'], ascending=True)

# static total games data frame
total_games = df2.groupby(['Name','PuzzleNum']).size().reset_index().groupby('Name').count().reset_index()

# static fails by difficulty data frame
avg_fails = df2.groupby(['Name','Difficulty']).agg({'Fails': 'mean'}).reset_index().sort_values(['Name'])

In [5]:
ranking_df

,Name,Fails,MeanWeeklyFails
2,Player3,208,5.333333
0,Player1,237,6.076923
3,Player4,240,6.153846
5,Player6,249,6.384615
1,Player2,250,6.410256
4,Player5,268,6.871795


In [6]:
total_games

,Name,PuzzleNum,0
0,Player1,78,78
1,Player2,78,78
2,Player3,78,78
3,Player4,78,78
4,Player5,77,77
5,Player6,78,78


## Work-in-Progress

In [8]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/dZVMbK.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Master Div
app.layout = html.Div(
    children=[
        html.H1(children='Wordle Ranking & Performance'),
        # Top Portion Div
        html.Div([
            # Top Left - Filters
            html.Div([
                # Checklist
                dcc.RadioItems(id='crossfilter-xaxis-column',
                              options=[{'label': i, 'value': i} for i in player_list],
                              value='Player1')
            ],
                style={'width':'49%'}
            ),
            # Top Right - Player Ranking by - Fails
            html.Div([
                dcc.Graph(id='bars-fail-total',
                          figure=px.bar(ranking_df,
                                        x="Fails",
                                        y="Name",
                                        color="Name",
                                       text_auto=True)
                         )
            ],
                style={'width':'49%'}
            )
        ],
            style={'display': 'flex'}
        ),
        # Bottom Portion Div
        html.Div([
            # Bottom Left - Summary
            html.Div([
                html.Div([
                    html.H2(id='total-games-played'),
                    html.H2(id='avg-fails'),
                    html.H2(id='avg-weekly-fails'), #style={'width':'49%'}),
                ]),
                # Bottom Right - Distribution by Player
                html.Div([dcc.Graph(id='bars-fail-distribution')], style={'width':'49%'})
            ],
                style={'display': 'flex'}
            )
        ])
    ],
    style={'width':'1120px', 'height':'630px'}
)

# update bars-fail-distribution
@app.callback(
    Output('bars-fail-distribution', 'figure'),
    Input('crossfilter-xaxis-column', 'value')
)
def update_distplot(selected_player):
    # input filters df onto a single name
    filtered_df = df2[ df2.Name == selected_player ]
    fig = px.histogram(
        filtered_df, 
        x="Fails", 
        # y="Counts", 
        color="Difficulty", 
        text_auto=True
        # barmode="group"
    )
    # now update the figture with a click
    fig.update_layout(transition_duration=250)

    return fig

# update total-games-played
@app.callback(
    Output('total-games-played', 'children'),
    Input('crossfilter-xaxis-column', 'value')
)
def update_total_games(selected_player):
    filtered_value = total_games[ total_games.Name == selected_player ]['PuzzleNum'].values
    return 'Total Games Played: {}'.format(filtered_value[0])

# update avg-fails
@app.callback(
    Output('avg-fails', 'children'),
    Input('crossfilter-xaxis-column', 'value')
)
def update_avg_fails(selected_player):
    filtered_value = avg_fails[ avg_fails.Name == selected_player]['Fails'].values
    return 'Mean Easy Game Fails {:.2f}, Mean Hard Game Fails {:.2f}'.format(filtered_value[0], filtered_value[1])

# update avg-weekly-fails
@app.callback(
    Output('avg-weekly-fails', 'children'), 
    Input('crossfilter-xaxis-column', 'value')
)
def update_avg_weekly_fails(selected_player):
    filtered_value = ranking_df[ ranking_df.Name == selected_player ]['MeanWeeklyFails'].values
    return 'Mean Fails Per Week {:.2f}'.format(filtered_value[0])

if __name__ == '__main__':
    # run better without 'mode="jupyterdash"'
    app.run_server(debug=True, threaded=True)

Dash app running on http://127.0.0.1:8050/
